In [ ]:
        import tensorflow as tf
        print(tf.config.list_physical_devices('GPU'))

In [ ]:
! python.exe -m pip install --upgrade pip
! pip install pandas
! pip install numpy
! pip install openpyxl
! pip install seaborn

In [1]:
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("\n*****Required libraries imported*****")


*****Required libraries imported*****


# Mention project raw data filename below

In [7]:
file_Name = 'cases_report_Nasscom1.0_All_(All States)_2024-02-13.csv'

In [8]:
def csvORexcel():
    try:
        path = "C:\\Python\\read\\"+file_Name
        if file_Name.split('.')[-1].startswith('c'):
            df = pd.read_csv(path)
            return df
        elif file_Name.split('.')[-1].startswith('x'):
            df = pd.read_excel(path)
            return df
    except FileNotFoundError:
        print("The file name {0} has not found".format(path))

fn = file_Name

In [9]:
data0 = csvORexcel()
data0.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,Case Organization,Case District,Citizen GUID,Citizen Name,DOB,Gender,Mobile,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name
0,EP01SVRD,28-Feb-2023,Digital Health ID card_MP,SH0009I2,Data complete,NaN,NaN,Not Suspected,MP - Nasscom Foundation,Damoh,PN01YXCW,Abhishek Rai,14-Oct-1999,M,8120331866,24.0,PG01RM4E,Rai,parash.sahu,Parasmani Sahu,Parasmani Sahu
1,EP01ZSED,28-Mar-2023,MP-Aadhar seeding to Samagra,DC00097P,Data complete,NaN,NaN,Not Suspected,MP - Nasscom Foundation,Damoh,PN026FQH,Naresh Kumar Ahirwar,10-Jul-1999,M,9625601462,24.0,PG01YRH3,Ahirwar,parash.sahu,Parasmani Sahu,Parasmani Sahu
2,EP0203IG,29-Mar-2023,E-Shram Card_MP,SH0009HI,Data complete,NaN,NaN,Not Suspected,MP - Nasscom Foundation,Damoh,PN01ZCK0,Parasmani Sahu,01-Jan-1997,M,9630483273,27.0,PG01S0GV,Sahu,parash.sahu,Parasmani Sahu,Parasmani Sahu
3,EP021FB4,03-Apr-2023,Registration of farmers for training on natura...,SH000A1G,Data complete,NaN,NaN,Not Suspected,MP - Nasscom Foundation,Damoh,PN0281HS,Lakhan Lal Pandey,01-Jan-1973,M,8889139573,51.0,PG020A2X,Pandey,parash.sahu,Parasmani Sahu,Parasmani Sahu
4,EP021M1G,04-Apr-2023,Digital Health ID card_MP,SH0009I2,Data complete,NaN,NaN,Not Suspected,MP - Nasscom Foundation,Damoh,PN0289XV,Sakshi Lodhi,20-Feb-2004,F,6266780959,19.0,PG020HEI,Lodhi,parash.sahu,Parasmani Sahu,Parasmani Sahu


In [10]:
states = {'AP':'Andhra Pradesh',
'AR':'Arunachal Pradesh',
'AS':'Assam',
'BR':'Bihar',
'BH':'Bihar',
'CT':'Chhattisgarh',
'CG':'Chhattisgarh',
'GA':'Goa',
'GJ':'Gujarat',
'HR':'Haryana',
'HP':'Himachal Pradesh',
'JH':'Jharkhand',
'KA':'Karnataka',
'KL':'Kerala',
'MP':'Madhya Pradesh',
'MH':'Maharashtra',
'MN':'Manipur',
'ML':'Meghalaya',
'MZ':'Mizoram',
'NL':'Nagaland',
'OR':'Odisha',
'PB':'Punjab',
'RJ':'Rajasthan',
'SK':'Sikkim',
'TN':'Tamil Nadu',
'TG':'Telangana',
'TR':'Tripura',
'UP':'Uttar Pradesh',
'UT':'Uttarakhand',
'WB':'West Bengal'}

# Data Cleaning

In [11]:
# Remove last row.
data0.drop(index = data0[data0.Createdon.isna()].index, inplace=True)

# Replace null values
data0['Scheme/Doc'].fillna('a', inplace=True)
data0['Citizen Name'].fillna('a', inplace=True)
data0['HD Name'].fillna('blank', inplace=True)
#data0.Mobile.fillna(0, inplace=True)

# Deleting records with status "Case Aborted" and "Application rejected"
data0 = data0[(data0['Status'] != 'Case Aborted') & (data0['Status'] != 'Application rejected')]

# Changing status values and keeping only "Open/Submit/BR"
data0['Status'] = data0['Status'].apply(lambda x: 'Open' if x == 'Data complete' else 'Submitted' if x=='Docket submitted' else "Benefit Received" if x=='Scheme/Document received' else x)

# Changing Case Organization values from state initials to full state name.
data0['Case Organization'] = data0['Case Organization'].apply(lambda x: states[x[:2]])

# Renaming column "Case Organiisation" & "Case District" to "State" & "Disctrict"
data0.rename(columns={"Case Organization":"State","Case District":"District"}, inplace=True)

# Change gender from initial letter to full form.
data0['Gender'] = data0['Gender'].apply(lambda x: 'Male' if x=='M' else 'Female' if x=='F' else 'Other' if x=='O' else x)

# Convert "Createdon", "Docket Submitted Date", "Benefit received Date" column data type to Datetime format
dt_col = ['Createdon', 'Docket Submitted Date', 'Benefit received Date', 'DOB']

for col in dt_col:
    try:
        data0[col] = pd.to_datetime(data0[col], format='mixed', errors='ignore')
    except KeyError:
        continue

data0.reset_index(inplace=True, drop=True)
data0.info()

AttributeError: 'NoneType' object has no attribute 'info'

# Translate local language district name to English

In [13]:
# Renaming Case District name from local language to english

data0['District'] = data0['District'].apply(lambda x: "Dohad" if x=="દોહદ" else "Haridwar" if x=="हरिद्वार" else "UDHAM SINGH NAGAR".title() if (x=="उदम सिंह नगर" or x=="Udam Singh Nagar" or x=="UDAM SINGH NAGAR")
                             else "Sitamarhi" if x=="सीतामढ़ी" else "Mahasamund" if (x=="महासमुंद" or x=="Mahasamand")
                             else "Rajgarh" if x=="राजगढ़" else "Muzaffarpur" if x=="मुजफ्फरपुर" else "Nawada" if x=="नवादा" else "Balrampur" if x=="बलरामपुर"
                             else "DAMOH".title() if x=="दमोह" else "Shravasti" if x=="श्रावस्ती" else "NARMADA".title() if x=="નર્મદા" else "Chhatarpur" if (x=="छतरपुर" or x=="Chhattarpur")
                             else 'East Singhbum' if x=='ईस्ट सिंघबम' else 'Chhindwara' if x=='छिंदवारा' else 'Jalna' if x=='जालना' else 'Dhule' if x=='धुळे' else "Dhanbad" if x=="धनबाद"
                             else 'Banas Kantha' if x=='બનાસ કાંઠા' else x.title())

# Renaming column "Case Organiisation" & "Case District" to "State" & "Disctrict"
data0.rename(columns={"Case Organization":"State","Case District":"District"}, inplace=True)

data0['District'].value_counts()

District
Gautam Buddha Nagar    31915
Name: count, dtype: int64

# DFL Schemes

- SH0009SW = Digital productivity Service_ Basic
- SH000BM6 = Digital productivity Service_Basic
- SH000AG6 = Digital Productivity Services_Advanced
- SH000A32 = Long Training on Digital & Financial Inclusion_Private
- SH0009SW = Short Training on Digital & Financial Inclusion_Private
- SH000AG6 = Digital Productivity Services and and employability training_Advanced

# Removing DFL data

In [14]:
# Adding column "Scheme Type" to differentiate Schemes and DFL
data0["Scheme Category"] = data0['Scheme/Doc GUID'].apply(lambda x: "DFL" if (x=="SH0009SW" or x=="SH000AG6" or x=="SH000A32" or x=="SH0009SW" or x=="SH000AG6" or x=="SH000BM6") else "E-Gov")

dfl = data0[(data0['Scheme/Doc GUID'] == 'SH0009SW') | (data0['Scheme/Doc GUID'] == 'SH000AG6') | (data0['Scheme/Doc GUID'] == 'SH000A32') | (data0['Scheme/Doc GUID']=='SH0009SW') | (data0['Scheme/Doc GUID']=='SH000AG6') | (data0['Scheme/Doc GUID']=='SH000BM6')]
data0 = data0[(data0['Scheme/Doc GUID'] != 'SH0009SW') & (data0['Scheme/Doc GUID'] != 'SH000AG6') & (data0['Scheme/Doc GUID'] != 'SH000A32') & (data0['Scheme/Doc GUID']!='SH0009SW') & (data0['Scheme/Doc GUID']!='SH000AG6') & (data0['Scheme/Doc GUID']!='SH000BM6')]
print("DFL Count={0}\nE-Gov Count={1}".format(len(dfl),len(data0)))

DFL Count=2743
E-Gov Count=29172


In [15]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29172 entries, 0 to 32208
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Case Id                     29172 non-null  object        
 1   Createdon                   29172 non-null  datetime64[ns]
 2   Scheme/Doc                  29172 non-null  object        
 3   Scheme/Doc GUID             29172 non-null  object        
 4   Status                      29172 non-null  object        
 5   Docket Submitted Date       28676 non-null  datetime64[ns]
 6   Benefit received Date       27427 non-null  datetime64[ns]
 7   HD Suspected Cases          29172 non-null  object        
 8   State                       29172 non-null  object        
 9   District                    29172 non-null  object        
 10  Citizen GUID                29172 non-null  object        
 11  Citizen Name                29172 non-null  object        


# Duplicate Column

In [16]:
# Creating column with name 'duplicate' by concatenation
data0['duplicate'] = data0['Scheme/Doc'] + data0['Citizen Name'] + data0['Mobile'] # Concatenation
data0['duplicate'] = data0['duplicate'].apply(lambda x: x.lower()) # Converting duplicate column in lower case because python considers ASCII values of each character while checking for duplicates.
dfl['duplicate'] = dfl['Scheme/Doc'] + dfl['Citizen Name'] + dfl['Mobile']
dfl['duplicate'] = dfl['duplicate'].apply(lambda x: x.lower())
df['duplicate'] = df['Citizen Name'] + df['Mobile']
df['duplicate'] = df['duplicate'].apply(lambda x: x.lower())
data0.Mobile = data0.Mobile.apply(lambda x : int(x.split('.')[0])) # Converting "Mobile" column to interget data type
dfl.Mobile = dfl.Mobile.apply(lambda x : int(x.split('.')[0])) # Converting "Mobile" column to interget data type

data0.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,State,District,Citizen GUID,Citizen Name,Gender,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name,Type of training channel ?,duplicate
0,EP0282YR,2023-06-07,E-Shram Card (Central),SH0009HD,Benefit Received,2023-06-13,2023-06-13,Not Suspected,Uttar Pradesh,Gautam Buddha Nagar,PN02ETYB,Gauri Shankar,Male,53.0,PG026NY9,Shankar,Mohit.Yogi1,Mohit Yogi,Rahul Rastogi,NaN,e-shram card (central)gauri shankarmohit yogi
1,EP0282Z8,2023-06-07,E-Shram Card (Central),SH0009HD,Benefit Received,2023-06-13,2023-06-13,Not Suspected,Uttar Pradesh,Gautam Buddha Nagar,PN02ETYU,Bisamillah Bisamillah,Male,60.0,PG026NYS,Bisamillah,Mohit.Yogi1,Mohit Yogi,Rahul Rastogi,NaN,e-shram card (central)bisamillah bisamillahmoh...
2,EP02831X,2023-06-07,Indira Gandhi National Widow Pension Scheme_UP,SH0000LL,Open,NaT,NaT,Not Suspected,Uttar Pradesh,Gautam Buddha Nagar,PN02EU1D,Anokhi Anokhi,Female,75.0,PG026O15,Anokhi,Mohit.Yogi1,Mohit Yogi,Rahul Rastogi,NaN,indira gandhi national widow pension scheme_up...
3,EP028328,2023-06-07,E-Shram Card (Central),SH0009HD,Benefit Received,2023-06-13,2023-06-13,Not Suspected,Uttar Pradesh,Gautam Buddha Nagar,PN02EU1L,Afroj Afroj,Male,53.0,PG026O1F,Afroj,Mohit.Yogi1,Mohit Yogi,Rahul Rastogi,NaN,e-shram card (central)afroj afrojmohit yogi
4,EP02832W,2023-06-07,E-Shram Card (Central),SH0009HD,Benefit Received,2023-06-13,2023-06-13,Not Suspected,Uttar Pradesh,Gautam Buddha Nagar,PN02EU28,Pooja Yadav,Female,21.0,PG026O20,Yadav,Mohit.Yogi1,Mohit Yogi,Rahul Rastogi,NaN,e-shram card (central)pooja yadavmohit yogi


# Duplicate Data

In [17]:
# Checking number of all duplicate records
duplicateData = data0[data0.duplicated(['duplicate'],keep=False)].sort_values('duplicate')
dflDuplicates = dfl[dfl.duplicated(['duplicate'],keep=False)].sort_values('duplicate')
duplicateData = pd.concat([duplicateData,dflDuplicates], ignore_index=True)
duplicateData.reset_index(inplace=True, drop = True)

try:
    duplicateData.drop(index=duplicateData.index[-1], inplace=True)
except IndexError:
    print(duplicateData.shape)

duplicateData.shape

(6113, 21)

# Unique Data

In [18]:
# Keeping uniques excluding duplicates
unique_data = data0.drop(index = data0[data0.duplicated(['duplicate'], keep='last')].index)
unique_data.reset_index(inplace=True, drop = True)
try:
    unique_data.drop(index=unique_data.index[-1], inplace=True)
except IndexError:
    print(unique_data.shape)

unique_data.shape

(25771, 21)

In [19]:
# Exporting data of all duplicate records to Excel file.
duplicateData.to_excel('C:\\Python\\export\\'+fn.split('.')[0]+'_duplicate.xlsx', index=False)
print('Export to Excel Success! Total {0} duplicate records exported.'.format(duplicateData.shape[0]))

Export to Excel Success! Total 6113 duplicate records exported.


In [20]:
# Exporting data of all duplicate records to Excel file.
with pd.ExcelWriter('C:\\Python\\export\\'+fn.split('.')[0]+'_uniques.xlsx') as writer:
    unique_data.to_excel(writer, sheet_name='Schemes data', index=False)
    if dfl.shape[0]<1:
        dfl.to_excel(writer, sheet_name='DFL data', index=False)
        
print('Export to Excel Success! Total {0} duplicate records exported.'.format(unique_data.shape[0]))

Export to Excel Success! Total 25771 duplicate records exported.
